### 데이콘 주차 수요 예측 데이터를 활용하여 머신러닝 학습 및 예측 수행

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df=pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
age_info=pd.read_csv('age_gender_info.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [3]:
df['임대보증금'][df['임대보증금']=='-']=np.nan
df['임대보증금']=df['임대보증금'].astype('float')
df['임대료'][df['임대료']=='-']=np.nan
df['임대료']=df['임대료'].astype('float')

test['임대보증금'][test['임대보증금']=='-']=np.nan
test['임대보증금']=test['임대보증금'].astype('float')
test['임대료'][test['임대료']=='-']=np.nan
test['임대료']=test['임대료'].astype('float')

df['전용면적'] = df['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

#### 전용면적 상하한적용(15~100)

In [4]:
idx = df[df['전용면적']>100].index
df.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = df[df['전용면적']<15].index
df.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

#### 결국 최종 제출을 위해서는 단지코드별 grouping이 필요함. 예제코드에서는 그러한 이유로 차이가
#### 발생하는 변수는 모두 버리고 모델에 학습을 시킴
#### 단지코드별 중복없는게 전용면적, 전용면적별세대수, 임대보증금, 임대료가 있음. 
#### 이경우 한 단지내 개별 집집마다 사용가능한 평수가 다르고 임대료 임대보증금이 다르다는 이야기이기
#### 때문에 이경우에는 최빈값을 따라가는게 상식적으로 옳다고 봄

In [5]:
# 결측치 대치 및 데이터프레임 재선언
df=df.fillna(0)


df1=df[df.columns.difference(['단지코드','등록차량수'])]

r_code=df.단지코드
df1_target=df['등록차량수']

test1=test[test.columns.difference(['단지코드'])]
r_code_t=test.단지코드

df1=pd.merge(df1,age_info,on='지역')
test=pd.merge(test,age_info,on='지역')

#### 표준화 스케일링 진행

In [6]:
from sklearn.preprocessing import StandardScaler

cat_col=[ i for i in df1.columns.values if df1[i].dtype == 'object']

num_col=[ j for j in df1.columns.values if df1[j].dtype in ['int64','float64'] ]

test_cat_col=[ i for i in test1.columns.values if test1[i].dtype == 'object']

test_num_col=[ j for j in test1.columns.values if test1[j].dtype in ['int64','float64'] ]




scaler=StandardScaler()

df1_cat=pd.get_dummies(df1.loc[:,cat_col])
df1_num=pd.DataFrame(scaler.fit_transform(df1.loc[:,num_col]),columns=num_col)

df1_sol=pd.concat([df1_cat,df1_num],axis=1)


test_cat=pd.get_dummies(test.loc[:,test_cat_col])
test_num=pd.DataFrame(scaler.fit_transform(test.loc[:,test_num_col]),columns=test_num_col)

test_sol=pd.concat([test_cat,test_num],axis=1)


len(df1_sol.columns)
len(test_sol.columns)

45

#### test의 범주형 변수의 수가 train데이터보다 작음 변수축소를 해야 모델에 적용가능

In [7]:
df1_sol=df1_sol.reindex(test_sol.columns.values,axis=1)
len(df1_sol.columns)

df1_sol=pd.concat([r_code,df1_sol,df1_target],axis=1)
test_sol=pd.concat([r_code_t,test_sol],axis=1)
#df1_target 이 차량등록수임


df1_sol.dtypes
f1 = lambda x : x.mode().median()

df1_sol=df1_sol.groupby('단지코드').agg(f1).reset_index()

df1_target=df1_sol.등록차량수

df1_sol=df1_sol.iloc[:,1:-1]

test_sol=test_sol.groupby('단지코드').agg(f1).reset_index()
test_sol=test_sol.iloc[:,1:]

#### XGBoost의 회귀모델을 사용하고 최적의 하이퍼파라미터 값을 찾기위해 gridsearch진행

In [8]:
from xgboost import XGBRegressor

xgbreg=XGBRegressor()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

params= {'max_depth':[2,8,15,20,40],'n_estimators':[10,50,100],'learning_rate':[0.1,0.2,0.3]}
gridcv=GridSearchCV(xgbreg,params,scoring='neg_mean_squared_error',cv=5)

gridcv.fit(df1_sol,df1_target)

gridcv.best_params_
# {'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 50}

xgbreg=XGBRegressor(learning_rate=0.1,max_depth=2,n_estimators=10)

scores=cross_val_score(xgbreg,df1_sol,df1_target,scoring='neg_mean_squared_error',cv=10)
scores=-scores
scores.mean() 

184893.38622383113

In [9]:
xgbreg.fit(df1_sol,df1_target)
y_pred=xgbreg.predict(test_sol)

##### 제출용 데이터 프레임 생성

In [10]:
submission=pd.read_csv('C:/Users/sanghyun/Desktop/dacon parking/sample_submission.csv')
submission['num']=y_pred


submission.to_csv('baseline.csv', index=False)